# NAIVE BAYES ALGORITHM 

##### The Naive Bayes algorithm is a simple and powerful classification technique based on Bayes’ Theorem. It’s called “naive” because it assumes that all features (attributes) are independent of each other, which is rarely true in real-world data—but surprisingly, it still works well in many scenarios.

💡 **How Naive Bayes Works (For Classification)**
- Calculate prior probabilities for each class.
- Calculate likelihoods for each feature value given each class.
- Multiply all probabilities together for each class.
- Choose the class with the highest posterior probability.

📦 **Types of Naive Bayes**
- Gaussian Naive Bayes – For continuous features (assumes normal distribution)
- Multinomial Naive Bayes – For discrete features (e.g., word counts in text)
- Bernoulli Naive Bayes – For binary features (e.g., 0 or 1 if word exists)

### Titanic Survival Based on Naive Bayes Algorithm

In [1]:
import pandas as pd 
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [2]:
### drop the unnecessary columns 
subdf = df[['Pclass', 'Sex', 'Age', 'Fare','Survived']]
subdf.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [3]:
inputs = subdf.drop('Survived', axis = 'columns')
target = subdf.Survived

### now lets convert the string (Sex column) to integer 

In [4]:
dummies = pd.get_dummies(inputs.Sex).astype(int)
dummies.head()

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


### now concat this dummies to the inputs dataframe

In [5]:
inputs = pd.concat([inputs , dummies], axis = 'columns')
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0
3,1,female,35.0,53.1000,1,0
4,3,male,35.0,8.0500,0,1


### now drop the sex columns and one of the dummy variable to avoid the dummy variable trap

In [6]:
final_inputs = inputs.drop(['Sex' , 'female'], axis = 'columns')
final_inputs.head()

,Pclass,Age,Fare,male
0,3,22.0,7.2500,1
1,1,38.0,71.2833,0
2,3,26.0,7.9250,0
3,1,35.0,53.1000,0
4,3,35.0,8.0500,1


### to find whether any columns has NaN value 

In [8]:
final_inputs.columns[final_inputs.isna().any()] ## this will show that which columns has NaN value 

Index(['Age'], dtype='object')

In [9]:
final_inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

### now to handle it we will use mean

In [10]:
mean = final_inputs['Age'].mean()
mean

np.float64(29.69911764705882)

In [12]:
final_inputs['Age'] = final_inputs['Age'].fillna(mean)
final_inputs.head(10)

,Pclass,Age,Fare,male
0,3,22.000000,7.2500,1
1,1,38.000000,71.2833,0
2,3,26.000000,7.9250,0
3,1,35.000000,53.1000,0
4,3,35.000000,8.0500,1
5,3,29.699118,8.4583,1
6,1,54.000000,51.8625,1
7,3,2.000000,21.0750,1
8,3,27.000000,11.1333,0
9,2,14.000000,30.0708,0


### now lets split our model datasets

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_inputs, target, test_size = 0.2)

In [53]:
len(X_train)

712

In [54]:
len(X_test)

179

## Training our model 

In [55]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [56]:
model.fit(X_train,y_train)

GaussianNB()

In [57]:
## check the accuracy 
model.score(X_test, y_test)

0.770949720670391

### now lets predict 

In [61]:
model.predict(X_test[:10])

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [60]:
y_test[:10]

777    1
616    0
64     0
116    0
790    0
232    0
588    0
566    0
845    0
843    0
Name: Survived, dtype: int64

In [64]:
model.predict_proba(X_test[:10]) ## checks the probability of surviving 

array([[0.34055763, 0.65944237],
       [0.9671172 , 0.0328828 ],
       [0.70631836, 0.29368164],
       [0.92054423, 0.07945577],
       [0.96621631, 0.03378369],
       [0.88450639, 0.11549361],
       [0.96323162, 0.03676838],
       [0.96113388, 0.03886612],
       [0.96523532, 0.03476468],
       [0.96645801, 0.03354199]])